<a href="https://colab.research.google.com/github/kiharalab/EM-GAN/blob/master/EM_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#EM-GAN: Improved Protein Structure Modeling Using Enhanced Cryo-EM Maps With 3D Deep Generative Networks
<a href="https://github.com/marktext/marktext/releases/latest">
   <img src="https://img.shields.io/badge/platform-Linux%20%7C%20Mac%20-green">
   <img src="https://img.shields.io/badge/Language-python3-green">
   <img src="https://img.shields.io/badge/Language-C-green">
   <img src="https://img.shields.io/badge/dependencies-tested-green">
   <img src="https://img.shields.io/badge/licence-GNU-green">
</a>      

EM-GAN is a computational tool, which enables capturing protein structure information from cryo-EM maps more effectively than raw maps. It is based on 3D deep learning. It is aimed to help protein structure modeling from cryo-EM maps. 

Cite : Sai Raghavendra Maddhuri Venkata Subramaniya, Genki Terashi & Daisuke Kihara. Improved Protein Structure Modeling Using Enhanced Cryo-EM Maps With 3D Deep Generative Networks. In submission (2022).
Copyright (C) 2022 Sai Raghavendra Maddhuri Venkata Subramaniya, Genki Terashi, Daisuke Kihara, and Purdue University.

License: GPL v3 for academic use. (For commercial use, please contact us for different licensing.)

Contact: Daisuke Kihara (dkihara@purdue.edu)

**We strongly suggest to use Google Chrome for EM-GAN Colab version. Other browsers such as Safari may raise errors when uploading or downloading files.**

If you are using other browsers, disabling tracking protection may help resolve the errors when uploading or downloading files.

For more details, see **<a href="#Instructions">Instructions</a>** of the notebook and checkout the **[EM-GAN GitHub](https://github.com/kiharalab/EM-GAN)**.

# About EM-GAN
![](https://github.com/kiharalab/EM-GAN/blob/master/data/git/architecture.png?raw=true)

#Overview of EM-GAN
An increasing number of biological macromolecules have been solved with cryo-electron microscopy (cryo-EM). Over the past few years, the resolutions of density maps determined by cryo-EM have largely improved in general. However, there are still many cases where the resolution is not high enough to model molecular structures with standard computational tools. If the resolution obtained is near the empirical border line (3 - 4 Å), improvement in the map quality facilitates improved structure modeling. Here, we report that protein structure modeling can often be substantially improved by using a novel deep learning-based method that prepares an input cryo-EM map for modeling. The method uses a three-dimensional generative adversarial network, which learns density patterns of high and low-resolution density maps.

# Instructions <a name="Instructions"></a>

**Quick start**

1. Connect to a gpu machine by clicking the right top button **"connect"** in the notebook, then we can run EM-GAN under GPU support.
2. Click the left running button in <a href="#Dependency">Install Dependencies</a> to install dependencies.
3. Upload your cryo-EM maps in mrc/map format by clicking the left running button in <a href="#Map">Upload Cryo EM maps</a>. If you want to use our example, please skip this step. Here we suggest user to upload a cryo-EM map with **spacing 1** to save the running time.<br>
Here is a simple instructions to do that via [ChimeraX](https://www.rbvi.ucsf.edu/chimerax/): <a name="ChimeraX"></a>
```
1 open your map via chimeraX. 
2 In the bottom command line to type command: vol resample #1 spacing 1.0
3 In ChimeraX, click "save", then choose "MRC density map(*.mrc)" in "Files of type", then in "Map" choose the resampled map, finally specify the file name and path to save.
4 Then you can use the resampled map to upload
```
5. Running EM-GAN by by clicking the left running button in <a href="#Running">Run EM-GAN</a>. 

# Run EM-GAN Online



In [1]:
#@title Install dependencies <a name="Dependency"></a>
#@markdown Please make sure the notebook is already connected to **GPU**, EM-GAN needs GPU support to run.<br>
#@markdown Click the right top button **"connect"**, then the notebook will automatically connect to a gpu machine
%cd /content
import urllib.request

#get_url= urllib.request.urlopen('https://kiharalab.org/emsuites/daq_count.php?pwd=daq_dklab')
#print(get_url)
!pip install mrcfile==1.2.0 
!pip install numpy>=1.19.4
!pip install numba>=0.52.0
!pip install torch>=1.6.0
!pip install scipy>=1.6.0
!pip install py3Dmol
!pip install matplotlib
!apt-get install libfftw3-dev
!rm -rf EM-GAN
!git clone https://github.com/kiharalab/EM-GAN --quiet
%cd EM-GAN 


/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libfftw3-bin libfftw3-long3 libfftw3-quad3 libfftw3-single3
Suggested packages:
  libfftw3-doc
The following NEW packages will be installed:
  libfftw3-bin libfftw3-dev libfftw3-long3 libfftw3-quad3 libfftw3-single3
0 upgraded, 5 newly installed, 0 to remove and 22 not upgraded.
Need to get 3,766 kB of archives.
After this operation, 21.2 MB of addit

In [2]:
#@title Input cryo-EM map <a name="Map"></a>
#@markdown If you choose to use author's example, please **skip** this. <br>
#@markdown Otherwise, please follow the instructions to upload your cryo-EM map file.   
#@markdown <br>
#@markdown <br>Here we suggest user to upload a cryo-EM map with **spacing 1** to save the running time. Detailed instructions with ChimeraX is <a href="#ChimeraX">ChimeraX resampling</a>
#@markdown <br> **Support file format: .mrc, .mrc.gz**
from google.colab import files
import os
import os.path
import re
import hashlib
import random
import string

rand_letters = string.ascii_lowercase
rand_letters = ''.join(random.choice(rand_letters) for i in range(20))
use_author_example = False #@param {type:"boolean"}
if not use_author_example:
  root_dir = os.getcwd()
  upload_dir = os.path.join(root_dir,rand_letters)
  if not os.path.exists(upload_dir):
    os.mkdir(upload_dir)
  os.chdir(upload_dir)
  map_input = files.upload()
  for fn in map_input.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(map_input[fn])))
    map_input_path = os.path.abspath(fn)
    print("Map save to %s"%map_input_path)
  os.chdir(root_dir)
else:
  print("you have chosen to use author's example, you can not upload map files any more.")
  map_input_path = os.path.join(os.getcwd(),"data")
  map_input_path = os.path.join(map_input_path,"2788.mrc")
  contour=0.16

Saving 6479_E.mrc to 6479_E.mrc
User uploaded file "6479_E.mrc" with length 24198208 bytes
Map save to /content/EM-GAN/engfwbrhrudmdsvkuuai/6479_E.mrc


In [3]:

from google.colab import files
import os
import os.path
import re
import hashlib
#@markdown Specify contour level <a name="Param"></a>
contour = '0.06' #@param {type:"string"}
#@markdown ```author recommended contour level for the input map. Using contour level will not have any impact on the result, but can reduce the computation time. ```
#@markdown <br>```default:0. Suggested Range: [0,author_contour]```


In [ ]:
#@title Run EM-GAN <a name="Running"></a>
#@markdown Please allow 30 mins on average to get the output, since 3D input processing and inferencing takes some time. 
#@markdown <br>Our running time is directly correlated to the size of the structures. For example, a map with 260 * 260 * 260 can take 2 hours to finish.
!git pull origin master
!rm -r data_dir/
old_path = os.path.join(os.getcwd(),"data")
old_path = os.path.join(old_path, "final.mrc")
!rm old_path
!chmod +x data_prep/HLmapData
!chmod +x MergeMap
import mrcfile
import numpy as np
import shutil
from numba import jit

@jit(nopython=True,nogil=True)
def interpolate_fast(data,data_new,size,iterator1,iterator2,iterator3,prev_voxel_size):
    for i in range(1, iterator1, 1):
        if(i%10==0):
            print("Finished resizing %d/%d"%(i,iterator1))
        for j in range(1, iterator2, 1):
            for k in range(1, iterator3, 1):
                count = [int(i / prev_voxel_size), int(j / prev_voxel_size), int(k / prev_voxel_size)]
                e1 = count[0] + 1
                e2 = count[1] + 1
                e3 = count[2] + 1

                if (count[0] >= size[0] - 1):  # or count[1]>=size[1]-1 or count[2]>=size[2]-1 ):
                    # print(count)
                    e1 = count[0]
                    continue
                if (count[1] >= size[1] - 1):
                    e2 = count[1]
                    continue
                if (count[2] >= size[2] - 1):
                    e3 = count[2]
                    continue
                diff1 = [i - count[0] * prev_voxel_size, j - count[1] *prev_voxel_size, k - count[2] * prev_voxel_size]
                diff2=[e1*prev_voxel_size-i,e2*prev_voxel_size-j,e3*prev_voxel_size-k]
                # print(diff)
                val1 = data[count[0], count[1], count[2]]
                val2 = data[e1, count[1], count[2]]
                val3 = data[e1, e2, count[2]]
                val4 = data[count[0], e2, count[2]]
                val5 = data[count[0], count[1], e3]
                val6 = data[e1, count[1], e3]
                val7 = data[e1, e2, e3]
                val8 = data[count[0], e2, e3]
                #val = (val1 + diff[0] * (val2 - val1) + diff[1] * (val4 - val1) + diff[2] * (val5 - val1) + diff[0] *
                #       diff[1] * (val1 - val2 + val3 - val4) + diff[0] * diff[2] * (val1 - val2 - val5 + val6) + diff[
                #           1] * diff[2] * (
                #               val1 - val4 - val5 + val8) + diff[0] * diff[1] * diff[2] * (
                #               val8 - val7 + val6 - val5 + val4 - val3 + val2 - val1))
                u1=diff1[0]
                u2=diff2[0]
                v1=diff1[1]
                v2=diff2[1]
                w1=diff1[2]
                w2=diff2[2]
                val=(w2*(v1*(u1*val3+u2*val4)+v2*(u1*val2+u2*val1))+w1*(v1*(u1*val7+u2*val8)+v2*(u1*val6+u2*val5)))/(w1+w2)/(v1+v2)/(u1+u2)
                data_new[i,j,k]=val
    return data_new#np.float32(data_new)

@jit(nopython=True,nogil=True)
def interpolate_fast_general(data,data_new,size,iterator1,iterator2,iterator3,
                             prev_voxel_size1,prev_voxel_size2,prev_voxel_size3):
    for i in range(1, iterator1, 1):
        if(i%10==0):
            print("Finished resizing %d/%d"%(i,iterator1))
        for j in range(1, iterator2, 1):
            for k in range(1, iterator3, 1):
                count = [int(i / prev_voxel_size1), int(j / prev_voxel_size2), int(k / prev_voxel_size3)]
                e1 = count[0] + 1
                e2 = count[1] + 1
                e3 = count[2] + 1

                if (count[0] >= size[0] - 1):  # or count[1]>=size[1]-1 or count[2]>=size[2]-1 ):
                    # print(count)
                    e1 = count[0]
                    continue
                if (count[1] >= size[1] - 1):
                    e2 = count[1]
                    continue
                if (count[2] >= size[2] - 1):
                    e3 = count[2]
                    continue
                diff1 = [i - count[0] * prev_voxel_size1, j - count[1] *prev_voxel_size2, k - count[2] * prev_voxel_size3]
                diff2 = [e1*prev_voxel_size1-i,e2*prev_voxel_size2-j,e3*prev_voxel_size3-k]
                # print(diff)
                val1 = data[count[0], count[1], count[2]]
                val2 = data[e1, count[1], count[2]]
                val3 = data[e1, e2, count[2]]
                val4 = data[count[0], e2, count[2]]
                val5 = data[count[0], count[1], e3]
                val6 = data[e1, count[1], e3]
                val7 = data[e1, e2, e3]
                val8 = data[count[0], e2, e3]
                #val = (val1 + diff[0] * (val2 - val1) + diff[1] * (val4 - val1) + diff[2] * (val5 - val1) + diff[0] *
                #       diff[1] * (val1 - val2 + val3 - val4) + diff[0] * diff[2] * (val1 - val2 - val5 + val6) + diff[
                #           1] * diff[2] * (
                #               val1 - val4 - val5 + val8) + diff[0] * diff[1] * diff[2] * (
                #               val8 - val7 + val6 - val5 + val4 - val3 + val2 - val1))
                u1=diff1[0]
                u2=diff2[0]
                v1=diff1[1]
                v2=diff2[1]
                w1=diff1[2]
                w2=diff2[2]
                val=(w2*(v1*(u1*val3+u2*val4)+v2*(u1*val2+u2*val1))+w1*(v1*(u1*val7+u2*val8)+v2*(u1*val6+u2*val5)))/(w1+w2)/(v1+v2)/(u1+u2)
                data_new[i,j,k]=val
    return data_new#np.float32(data_new)

def interpolate_slow(data,data_new,size,iterator1,iterator2,iterator3,prev_voxel_size):
    for i in range(1, iterator1, 1):
        if(i%10==0):
            print("Finished resizing %d/%d"%(i,iterator1))
        for j in range(1, iterator2, 1):
            for k in range(1, iterator3, 1):
                count = [int(i / prev_voxel_size), int(j / prev_voxel_size), int(k / prev_voxel_size)]
                e1 = count[0] + 1
                e2 = count[1] + 1
                e3 = count[2] + 1

                if (count[0] >= size[0] - 1):  # or count[1]>=size[1]-1 or count[2]>=size[2]-1 ):
                    # print(count)
                    e1 = count[0]
                    continue
                if (count[1] >= size[1] - 1):
                    e2 = count[1]
                    continue
                if (count[2] >= size[2] - 1):
                    e3 = count[2]
                    continue
                diff1 = [i - count[0] * prev_voxel_size, j - count[1] *prev_voxel_size, k - count[2] * prev_voxel_size]
                diff2=[e1*prev_voxel_size-i,e2*prev_voxel_size-j,e3*prev_voxel_size-k]
                # print(diff)
                val1 = data[count[0], count[1], count[2]]
                val2 = data[e1, count[1], count[2]]
                val3 = data[e1, e2, count[2]]
                val4 = data[count[0], e2, count[2]]
                val5 = data[count[0], count[1], e3]
                val6 = data[e1, count[1], e3]
                val7 = data[e1, e2, e3]
                val8 = data[count[0], e2, e3]
                #val = (val1 + diff[0] * (val2 - val1) + diff[1] * (val4 - val1) + diff[2] * (val5 - val1) + diff[0] *
                #       diff[1] * (val1 - val2 + val3 - val4) + diff[0] * diff[2] * (val1 - val2 - val5 + val6) + diff[
                #           1] * diff[2] * (
                #               val1 - val4 - val5 + val8) + diff[0] * diff[1] * diff[2] * (
                #               val8 - val7 + val6 - val5 + val4 - val3 + val2 - val1))
                u1=diff1[0]
                u2=diff2[0]
                v1=diff1[1]
                v2=diff2[1]
                w1=diff1[2]
                w2=diff2[2]
                val=(w2*(v1*(u1*val3+u2*val4)+v2*(u1*val2+u2*val1))+w1*(v1*(u1*val7+u2*val8)+v2*(u1*val6+u2*val5)))/(w1+w2)/(v1+v2)/(u1+u2)
                data_new[i,j,k]=val
    return data_new

def Reform_Map_Voxel(map_path,new_map_path):
    print("here")
    if not os.path.exists(new_map_path):
        with mrcfile.open(map_path,permissive=True) as mrc:
            prev_voxel_size=mrc.voxel_size
            #assert len(prev_voxel_size)==3

            if not(prev_voxel_size['x']==prev_voxel_size['y'] and prev_voxel_size['x']==prev_voxel_size['z']):
                print("Grid size of different axis is different, please specify --resize=1 in command line to call another slow process to deal with it!")
                exit(1)
            prev_voxel_size=float(prev_voxel_size['x'])
            nx, ny, nz, nxs, nys, nzs, mx, my, mz =\
                mrc.header.nx, mrc.header.ny, mrc.header.nz, \
                mrc.header.nxstart, mrc.header.nystart, mrc.header.nzstart,\
                mrc.header.mx, mrc.header.my, mrc.header.mz
            orig = mrc.header.origin
            print("Origin:",orig)
            print("Previous voxel size:",prev_voxel_size)
            data = mrc.data
            data = np.swapaxes(data, 0, 2)
            size = np.shape(data)
            if (prev_voxel_size==1):
                shutil.copy(map_path,new_map_path)
                return new_map_path
            if (prev_voxel_size < 1):
                print("Grid size is smaller than 1, please specify --resize=1 in command line to call another slow process to deal with it!")
                exit(1)
            it_val1 = int(np.floor(size[0] * prev_voxel_size))
            it_val2 = int(np.floor(size[1] * prev_voxel_size))
            it_val3 = int(np.floor(size[2] * prev_voxel_size))
            print("Previouse size:",size," Current map size:",it_val1,it_val2,it_val3)
            data_new = np.zeros([it_val1,it_val2,it_val3])
            data_new[0, 0, 0] = data[0, 0, 0]
            data_new[it_val1 - 1, it_val2 - 1, it_val3 - 1] = data[
                size[0] - 1, size[1] - 1, size[2] - 1]
            #iterator = Value('i', it_val)
            #s = Value('d', float(prev_voxel_size))
            #pool = Pool(3)
            #out_1d = pool.map(interpolate,enumerate(np.reshape(data_new, (iterator.value * iterator.value * iterator.value,))))
            #data_new = np.array(out_1d).reshape(iterator.value, iterator.value, iterator.value)
            try:
                data_new=interpolate_fast(data,data_new,size,it_val1,it_val2,it_val3,prev_voxel_size)
            except:
                data_new = np.zeros([it_val1, it_val2, it_val3])
                data_new[0, 0, 0] = data[0, 0, 0]
                data_new[it_val1 - 1, it_val2 - 1, it_val3 - 1] = data[
                    size[0] - 1, size[1] - 1, size[2] - 1]
                data_new = interpolate_slow(data, data_new, size, it_val1,it_val2,it_val3, prev_voxel_size)
            data_new = np.swapaxes(data_new, 0, 2)
            data_new=np.float32(data_new)
            mrc_new = mrcfile.new(new_map_path, data=data_new, overwrite=True)
            vsize = mrc_new.voxel_size
            vsize.flags.writeable = True
            vsize.x = 1.0
            vsize.y = 1.0
            vsize.z = 1.0
            mrc_new.voxel_size = vsize
            mrc_new.update_header_from_data()
            mrc_new.header.nxstart = nxs * prev_voxel_size
            mrc_new.header.nystart = nys * prev_voxel_size
            mrc_new.header.nzstart = nzs * prev_voxel_size
            mrc_new.header.mapc = mrc.header.mapc
            mrc_new.header.mapr = mrc.header.mapr
            mrc_new.header.maps = mrc.header.maps
            mrc_new.header.origin = orig
            mrc_new.update_header_stats()
            mrc.print_header()
            mrc_new.print_header()
            mrc_new.close()
            del data
            del data_new
           # del out_1d
    return new_map_path

new_map_path = os.path.join(os.getcwd(),"data")
new_map_path = os.path.join(new_map_path,"final.mrc")
Reform_Map_Voxel(map_input_path,new_map_path)
!data_prep/HLmapData -a "{new_map_path}" -b "{new_map_path}" -A  "{contour}" -B "{contour}" -w 12 -s 4 >  protein_trimmap
!python data_prep/generate_input.py protein_trimmap protein_data ./data_dir/
!python test.py --res_blocks=5 --batch_size=128 --in_channels=32 --G_path=model/G_model --D_path=model/D_model --dir_path=data_dir/
!python sr_dataprep.py
!python avg_model.py

From https://github.com/kiharalab/EM-GAN
 * branch            master     -> FETCH_HEAD
Already up to date.
rm: cannot remove 'old_path': No such file or directory
here
2270
2270
18
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
0
50431488
./data_dir
protein
./data_dir/flag_dir/protein.flag
protein done
./data_dir/sr_hr_lr/
2270
./MergeMap -l ./tmpF
# #model= 2270
#reading.. ./data_dir/sr_hr_lr/protein_0_SR.situs
#Pos=20 120 96
#Pos=20 120 96
#reading.. ./data_dir/sr_hr_lr/protein_1_SR.situs
#Pos=20 124 92
#Pos=20 124 92
#reading.. ./data_dir/sr_hr_lr/protein_2_SR.situs
#Pos=20 124 96
#Pos=20 124 96
#reading.. ./data_dir/sr_hr_lr/protein_3_SR.situs
#Pos=20 128 96
#Pos=20 128 96
#reading.. ./data_dir/sr_hr_lr/protein_4_SR.situs
#Pos=24 112 80
#Pos=24 112 80
#reading.. ./data_dir/sr_hr_lr/protein_5_SR.situs
#Po

In [ ]:
#@title Download EM-GAN Output <a name="Download"></a>
#@markdown Download modified EM map as Merged.mrc
from google.colab import files
import os
download_path = os.path.join(os.getcwd(),"Merged.mrc")
files.download(download_path)

## Citation: <a name="Citation"></a>

Sai Raghavendra Maddhuri Venkata Subramaniya, Genki Terashi & Daisuke Kihara. Improved Protein Structure Modeling Using Enhanced Cryo-EM Maps With 3D Deep Generative Networks. In submission (2022).